# Display objects/screens in the object list

In [1]:
import bigpemu_debug as bigp
import struct

"""
Code starts 0x00192000
"""
locations = {
    # The address to start at
    # The number of bytes to fetch
    "ddlist":     (0xEF9A,8),
}

def send_hex_dump_cmd_raw(from_addr,dump_len,responses=1):
    from_addr = struct.pack('<I', from_addr)
    dump_len = struct.pack('<H', dump_len)

    hex_dump = \
    b"\x0c\x00\x00\x00\x10\x00\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00" \
    + from_addr + b"\x00\x00\x00\x00" + dump_len + b"\x00\x00\x00\x00\x00\x00"

    s.sendall(hex_dump)
    response = bigp.get_response(s,responses)
    return response[0][2][24:] # Snip header

def send_hex_dump_cmd(location):
    from_addr,dump_len = locations[location]
    from_addr = struct.pack('<I', from_addr)
    dump_len = struct.pack('<H', dump_len)

    hex_dump = \
    b"\x0c\x00\x00\x00\x10\x00\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00" \
    + from_addr + b"\x00\x00\x00\x00" + dump_len + b"\x00\x00\x00\x00\x00\x00"

    s.sendall(hex_dump)
    response = bigp.get_response(s,1)
    return response[0][2][24:] # Snip header

def dump_standard(hex_dump,WIDTH=32):
    mem = '\n'.join([
        ','.join(['{:02X}'.format(x) for x in hex_dump[l:l+WIDTH]]) 
        for l in range(0,len(hex_dump),WIDTH)
    ])
    return mem

def hoover_up():
    resp = []
    while True:
        try:
            data = bigp.recv_msg(s)
        except (socket.error):
            break
        resp += [data]
    return resp


        

## Launch BigPEmu and connect to the debug server

In [8]:
import socket
import fcntl,os
import time

# Launch bigpemu and wait a moment.
os.system("wine ./BigPEmu_v118-DEV/BigPEmuDev.exe ../tempest2k/t2000.abs &")
time.sleep(5)

HOST = "127.0.0.1"  # The server's hostname or IP address
PORT = 44123  # The port used by the server

# Connect to the debug server
try:
    s.shutdown(socket.SHUT_RDWR)
except:
    pass
try:
    s.close()
except:
    pass
time.sleep(1)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
fcntl.fcntl(s, fcntl.F_SETFL, os.O_NONBLOCK)


0

In [9]:
import bigpemu_debug as bigp
import struct

"""
Set a breakpoint in 'scar' in antelope.gas at f035e4.
"""
breakpoint = b"\x11\x00\x00\x00\x30\x00\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00" \
            b"\x01\x00\x00\x00\x00\x00\x00\x00\xe4\x35\xf0\x00\x00\x00\x00\x00" \
            b"\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00" \
            b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"

s.sendall(breakpoint)
response = hoover_up()


In [33]:
resume = b"\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
s.sendall(resume)
response = hoover_up()


In [32]:
removeall = b"\x11\x00\x00\x00\x08\x00\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00" \
            b"\x00\x00\x00\x00\x00\x00\x00\x00"
s.sendall(removeall)
response = hoover_up()


In [86]:
gpu_addr = 0x80c3d4
response = hoover_up()
hex_dump = send_hex_dump_cmd_raw(gpu_addr,0x10)
print(dump_standard(hex_dump,WIDTH=4))


00,00,84,7A
6B,00,21,D0
20,79,00,00
84,80,20,39


In [16]:
gpu_addr = 0xF03F60
hex_dump = send_hex_dump_cmd_raw(gpu_addr,0x3c)
print(dump_standard(hex_dump,WIDTH=4))


00,13,48,00
00,00,01,80
00,00,00,F0
00,00,01,F4
00,00,01,F4
00,00,00,00
00,C0,00,00
00,78,00,00
00,00,00,00
00,F8,00,00
01,10,00,00
00,00,00,00
00,00,00,00
00,01,3E,B8
00,01,79,22


In [88]:
from_addr = 0xF02200
dump_len = 0x3c
from_addr = struct.pack('<I', from_addr)
dump_len = struct.pack('<H', dump_len)

hex_dump = \
b"\x0c\x00\x00\x00\x10\x00\x00\x00\x15\x00\x00\x00\x00\x00\x00\x00" \
+ from_addr + b"\x00\x00\x00\x00" + dump_len + b"\x00\x00\x00\x00\x00\x00"

s.sendall(hex_dump)
response = bigp.get_response(s,1)
print(response)



[(bytearray(b'\x07\x00\x00\x00T\x00\x00\x00K\x00\x00\x00\x00\x00\x00\x00'), 84, bytearray(b'\x15\x00\x00\x00\x00\x00\x00\x00`?\xf0\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x01\x80\x00\x00\x00\xf0\x00\x00\x01\xf4\x00\x00\x01\xf4\x00\x00\x00\xff\x00\xc0\x00\x00\x00x\x00\x00\x00\x00\x00\x00\x00\xf8\x00\x00\x00\x00\x00f\x00\x00\x00"\x00\x00\x00\x00\x00\x01\x07\xa8\x00\x01Z\xd4'))]


# Rough Work

In [81]:
hex(0xfff - 4088)

'0x7'